# Tutorial 5: Langchain Agent

In this tutorial we explore how to combine Langchain's modular interfaces with a robust C++ runtime, that is portable to mobile also.

In [ ]:
import textwrap
import langchain
import generative_computing as genc

## Define Tools
WolframAlpha is a Tool, therefore we wrap it under a LangChain Tool interface.*italicized text*

In [ ]:
class WolframAlpha(genc.interop.langchain.CustomTool):
  """A specific implementation of CustomTool, including an appid for identification."""

  def __init__(self, appid: str):
    super().__init__()
    self.appid = appid
    self.name = "WolframAlpha"
    self.description = (
        "https://products.wolframalpha.com/api/documentation, one capability is"
        " to evaluate math expression."
    )
    # Extract math equation from a text, then calls WolfraAlpha
    self.computation = (
        genc.interop.langchain.CustomChain()
        | genc.authoring.create_custom_function("/react/extract_math_question")
        | genc.authoring.create_wolfram_alpha(appid)
        # Template Engine will extract the result from response JSON
        | genc.authoring.create_inja_template(
            "{% if queryresult.pods"
            " %}{{queryresult.pods.0.subpods.0.plaintext}}{% endif %}"
        )
    )

## Define Agent with Tool

In this example, we already moved the reusable agent/reasoning code into a LangChain interopertor, so you no longer need to write that.

In this section we explore how to take the prepackaged Agent to speed up your dev process.

In [ ]:
class MathToolAgent(genc.interop.langchain.CustomAgent):
  """An agent that combines ReAct reasoning loop with WolframAlpha tools."""

  def __init__(self, appid: str):

    system_instruction = textwrap.dedent("""
    Solve a question answering task with interleaving Thought, Action, Observation steps
    Thought can reason about the current situation
    Action can be only two types:
    (1) Math[query], Useful for when you need to solve math problems.
    (2) Finish[answer], which returns the answer as a number terminates.
    Here's an example:
    Question: what is the result of power of 2 + 1?
    Thought: power of 2
    Action: Math[pow(2)]
    Observation: 4
    Thought: I can now process the answer.
    Action: Math[4+1]
    Observation: 5
    Thought: Seems like I got the answer.
    Action: Finish[5]
    Please do it step by step.
    Question: {question}""")

    prompt = langchain.prompts.PromptTemplate(
        input_variables=["question"], template=system_instruction
    )
    tools = [WolframAlpha(appid)]
    # TODO(b/315872721): replace with CreateModelWithConfig
    llm = genc.interop.langchain.CustomModel(uri="/ai_studio/gemini_pro")

    # Init langchain.agents.agent.Agent with components backed by C++ runtime.
    super().__init__(
        llm_chain=langchain.chains.LLMChain(llm=llm, prompt=prompt),
        allowed_tools=[tool.name for tool in tools],
        tools_list=tools,
        max_iterations=5,
    )

## Compile & Run it

In [ ]:
wolfram_app_id="<>"
agent = MathToolAgent(wolfram_app_id)

portable_ir = genc.interop.langchain.create_computation(agent)

# To run it localy
runner = genc.runtime.Runner(portable_ir)
result = runner(
    "what is the result of (square root of 4) + 3 to the power of 2 + 3 *(8 +"
    " 4) / 2 - 7"
)

## Intro to next tutorial: Porting it to Mobile

IR stands for intermediate representation. GenC transforms the agent into a computation schema or proto, which is platform agnostic. Therefore we can directly load it on a platform with GenC runtime. The next tutorial will show how this is done.

In [ ]:
# Let's take a peak of how the IR or computation proto looks like
portable_ir